## Hente rådata fra Frost API-et hos Met.no
<div style="border: 2px solid yellow; padding: 10px; border-radius: 5px; width: 95%; ">
<style>
p{
    line-height: 1.5;
}
</style>
Her i denne koden har vi hentet værdata fra Meteorologisk Institutt. Vi har valgt å hente data herfra fordi vi vet at Meteorologisk Institutt er et annerkjent norsk statlig forvaltningsorgan, med ansvar for den offentlige meteorologiske tjenesten i Norge. I og med at det er driftet av staten har det en viss integeritet og pålitelighet. 

Vi har fått en "client-id" som gir oss tilgang til å hente rådata fra Frost-API-et til Meteorologisk Institutt. I koden under henter vi da rådata mellom "start_dato" og "slutt_dato" og lagrer det i csv-filen "VaerData.csv" for visualisering. 
</div>


In [9]:
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import json
import sys 

sys.path.append('../src') # Legger til src-mappen i systemstien for å importere DataBehandler
from data_behandling import DataBehandler

load_dotenv()

# Henter Client-id fra .env filen. Client ID for API-et Frost hos Meteorologisk institutt
client_id = os.getenv("client_id")

with open('../data/Byer.json', 'r') as f:
    byer = json.load(f)

# Oppretter en string med byer vi vil se på, velger parametrene og tidsperiode
byerAsString = ','.join(byer.values())
element_id = "mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)"  # Temperatur, nedbør og vindhastighet
start_dato = "2022-01-01"  
slutt_dato = "2023-01-01"  
with open("../src/tidsintervall.py", "w") as f:
    f.write(f"start_dato = '{start_dato}'\n")
    f.write(f"slutt_dato = '{slutt_dato}'\n")

# Endpoint og parametere for forespørselen
endpoint = 'https://frost.met.no/observations/v0.jsonld'
parameters = {
    "sources": byerAsString,
    "elements": element_id,
    "referencetime": f"{start_dato}/{slutt_dato}",
    "timeoffsets": "default",
    "timeresolutions": "P1D",
    "levels": "default"
}
print(f"Parametere: {parameters}")

# Sender forespørsel til Frost API hos meterelogisk institutt
response = requests.get(endpoint, params=parameters, auth=(client_id, ''))

# Undersøker om forespørselen var vellykket
if response.status_code != 200:
    print(f"Error! Status code: {response.status_code}")
    print(f"Message: {response.json().get('error', {}).get('message', 'Ingen melding gitt')}")
    print(f"Reason: {response.json().get('error', {}).get('reason', 'Ingen årsak gitt')}")
    exit()

# Henter JSON-data
json_data = response.json()
if 'data' not in json_data:
    print("Ingen data funnet i forespørselen.")
    print(json_data)
    exit()

# Oppretter en For-løkke for å gjøre om data til en DataFrame
data = json_data['data']
rows = [] 
for item in data:
    for observation in item['observations']:
        row = {
            'sourceId': item['sourceId'].split(':')[0],  
            'referenceTime': item['referenceTime'],
            'elementId': observation['elementId'],
            'value': observation['value'],
            'unit': observation['unit'],
            'codequality': observation.get('qualityCode', '')  
        }
        rows.append(row)


df = pd.DataFrame(rows)

behandler = DataBehandler(df, byer)

# Lagrer dataene i en CSV-fil "VaerData.csv".
behandler.lagre_rådata()

print("\nDataene er hentet fra Meterologisk Institutt.")

Parametere: {'sources': 'SN68230,SN18700,SN50540', 'elements': 'mean(air_temperature P1D),sum(precipitation_amount P1D),mean(wind_speed P1D)', 'referencetime': '2022-01-01/2023-01-01', 'timeoffsets': 'default', 'timeresolutions': 'P1D', 'levels': 'default'}

Rådata lagret i ../data/VaerData.csv

Dataene er hentet fra Meterologisk Institutt.
